In [1]:
import numpy as np
import h5py
from keras.models import Model
from keras import models
from keras import layers
from keras import Input
from keras import optimizers
from keras.layers import Dropout
from keras.layers import BatchNormalization
import pydot
import graphviz
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model
from keras.models import load_model
#channel_model=['B','C','D','E','F']
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

channel_model=['F']

for model_idx in channel_model:
    filepath='./wlan_train_set_model_%s.mat' % model_idx
    f=h5py.File(filepath,'r')
    train_data_set=np.array(f['Train_data'][:])
    cfo_label_set=np.array(f['Train_cfo_label'][:])/1e3
    train_data=train_data_set[0:90000]
    cfo_label=cfo_label_set[0:90000]
    valid_data=train_data_set[90000:100000]
    valid_label=cfo_label_set[90000:100000]
    input_data=Input(shape=(11,864,1))
    x=layers.Conv2D(32,(11,1),padding='same',activation='relu',kernel_initializer='he_normal')(input_data)
    x=layers.MaxPooling2D((1,4),padding='same')(x)
    x_skip1=layers.Conv2D(128,(1,1),strides=(1,2),padding='same',activation='relu',kernel_initializer='he_normal')(x)
    x=layers.Conv2D(64,(11,1),padding='same',activation='relu',kernel_initializer='he_normal')(x)
    x=layers.MaxPooling2D((1,2),padding='same')(x)
    x=layers.Conv2D(128,(11,1),padding='same',activation='relu',kernel_initializer='he_normal')(x)
    x=layers.add([x_skip1,x])
    x=layers.MaxPooling2D((1,4),padding='same')(x)
    x_skip2=layers.Conv2D(512,(1,1),strides=(1,2),padding='same',activation='relu',kernel_initializer='he_normal')(x)
    x=layers.Conv2D(256,(11,1),padding='same',activation='relu',kernel_initializer='he_normal')(x)
    x=layers.MaxPooling2D((1,2),padding='same')(x)
    x=layers.Conv2D(512,(11,1),padding='same',activation='relu',kernel_initializer='he_normal')(x)
    x=layers.add([x_skip2,x])
    x=layers.Flatten()(x)
    #x=layers.Dense(512,activation='relu',kernel_initializer='he_normal')(x)
    x=layers.Dense(1,name='cfo')(x)
    model= Model(input_data,x)
    model.compile(optimizer=optimizers.Adam(lr=0.001),
                loss={'cfo':'mse'},
                loss_weights={'cfo':1},
                metrics={'cfo':'mse'})
    model.fit(train_data,{'cfo':cfo_label},epochs=100,batch_size=128,validation_data=(valid_data,valid_label))
    
    model_name='wlan_CFO_model_%s_v2.h5' % model_idx
    model.save(model_name)

Using TensorFlow backend.




Train on 90000 samples, validate on 10000 samples
Epoch 1/100
21632/90000 [======>.......................] - ETA: 1:38 - loss: 1610.4359 - mse: 1610.4354

KeyboardInterrupt: 

In [2]:
train_data.shape

(95000, 11, 864, 1)